In [24]:
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly import offline

# Parameters
n_stars = 1000  # Number of stars
number_of_steps = 100
dimensions = 3  # X, Y, Z
filename = 'data/sim.dat'  # Unformatted binary file from Fortran

# Function to load unformatted binary data (positions)
def load_positions(filename, step, n_stars):
    # Open the file in binary mode and read the data corresponding to one step
    with open(filename, 'rb') as f:
        # Skip all previous data (each step contains 3*n_stars floats)
        f.seek(step * n_stars * dimensions * 8)  # 8 bytes per float (real*8 in Fortran)
        positions = np.fromfile(f, dtype=np.float64, count=n_stars * dimensions)
        positions = positions.reshape((n_stars, dimensions))
    return positions

# Create the initial figure
fig = make_subplots(rows=1, cols=1)

# Set the axis limits
fig.update_layout(
    scene=dict(
        xaxis=dict(range=[-1.5, 1.5], title='X'),
        yaxis=dict(range=[-1.5, 1.5], title='Y'),
        zaxis=dict(range=[-1.5, 1.5], title='Z')
    ),
    title="Star Positions Evolution",
    margin=dict(l=0, r=0, b=0, t=50)
)

# Create scatter plot for the stars
trace = go.Scatter3d(
    x=[], y=[], z=[],  # Empty data initially
    mode='markers',
    marker=dict(size=2, color='blue')
)

# Add the initial trace to the plot
fig.add_trace(trace)

# Number of steps for the simulation

# Define the update function for the animation
def update_plot(frame):
    positions = load_positions(filename, frame, n_stars)  # Load positions for this frame
    fig.data[0].x = positions[:, 0]  # X positions
    fig.data[0].y = positions[:, 1]  # Y positions
    fig.data[0].z = positions[:, 2]  # Z positions
    return fig.data

# Create the animation using the frames
frames = [go.Frame(
    data=[go.Scatter3d(
        x=load_positions(filename, i, n_stars)[:, 0],
        y=load_positions(filename, i, n_stars)[:, 1],
        z=load_positions(filename, i, n_stars)[:, 2],
        mode='markers',
        marker=dict(size=2, color='blue')
    )],
    name=str(i)
) for i in range(number_of_steps)]

# Add frames to the figure
fig.frames = frames

# Set the animation settings
fig.update_layout(
    updatemenus=[dict(
        type='buttons',
        x=0.1,
        y=-0.1,
        showactive=False,
        buttons=[dict(label='Play',
                      method='animate',
                      args=[None, dict(frame=dict(duration=16, redraw=True), fromcurrent=True)])]
    )]
)

# Show the plot in the browser
offline.plot(fig, filename='star_positions_animation.html')
# fig.show()


'star_positions_animation.html'